In [1]:
import requests
import urllib.request
from urllib.parse import quote
import numpy as np
from tqdm import tqdm
#import contextlibr
import collections
import os
import json
import tqdm
from bs4 import BeautifulSoup as bs
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import collections
import pandas as pd

### Set up spotify scrapping functions

In [17]:
token = 'BQBHUEbySKuXb-EW5f9LvtaiUWMpCaZCAuN3pHpH5Qrd_7wfc7rLRVJ0RMWCW1BKaoom7sWXGkpDWZAQ13GCkYd5NCT71TviKwj2l7aaizuO4w_Knk4js82N7AUsKCGeSUWkiSdLC2Yg0Xl_Fe1MzGIgkwk'

header = {
    'Authorization': 'Bearer ' + token,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
}

all_features = ['danceability',      \
                'energy',            \
                'speechiness',       \
                'acousticness',      \
                'tempo',             \
                'instrumentalness',  \
                'liveness',          \
                'valence']
all_genres = ['contemporary-rnb', 'british-rnb', 'alternative-rnb', 'new-jack-swing-rnb', 'new-orleans-rnb', 'jump-blues', 
             'blue-eyed-soul', 'early-rnb', 'motown-rnb']

In [3]:
final_json = []
intermediary_results = []

In [10]:
def song_features(song_id):
    query = 'audio-features/' + str(song_id)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()

def songs_features(songs_ids):
    query = 'audio-features/?ids=' + ','.join(songs_ids)
    r = requests.request('get', 'https://api.spotify.com/v1/' + query, headers=header)
    return r.json()['audio_features']


def get_rateYourMusic_tuples(filenames, table_index=3) :
    
    #song year, artist and title
    searches = []
    
    for f in filenames :

        res = bs(open(f),"html5lib")
        for t in res.findAll(attrs={'class': "mbgen"})[table_index].findAll("tr")[1:] :
                        
            if len(t.findAll("td")) >= 3 :
                
                
            
                year = t.findAll("td")[0].text
                artist = t.findAll("td")[1].text
                title = t.findAll("td")[2].text

                if '/' in title :
                    searches.append([year, artist, title.split('/')[0].strip()])
                    searches.append([year, artist, title.split('/')[1].strip()])

                else :
                    searches.append([year, artist, title])

    return searches

def search_tuples(genre, searches, nb_songs=20) :
    
    found = 0
    found_tuples = 0
    all_info = []

    for e, s in tqdm.tqdm(enumerate(searches)) :
        result  = requests.request('get', 'https://api.spotify.com/v1/search?q=album:'+ quote(s[2]) + "%20artist:" + quote(s[1]) + "&type=track", headers=header).json()
        time.sleep(0.3)
        items = result['tracks']['items'][:nb_songs]
        
        
        #We found the song
        if len(items) != 0 and ('error' not in result.keys()) :           
            
            album_type = result['tracks']['items'][0]['album']['album_type']
            
            #the song does not come from a compilation
            if album_type != 'compilation' :
                
                album       = items[0]['album']['name']
                release     = items[0]['album']['release_date']
                track_ids   = [c['id']   for c in items]
                track_names = [c['name'] for c in items]
                
                features = [(f,e) for e,f in enumerate(songs_features(track_ids)) if ('error' not in f.keys())]
                feature_indices = [i[1] for i in features]
                features = [i[0] for i in features]
                feature_ids = [track_ids[i] for i in feature_indices]
                
                

                if len(features) != 0 :

                    found += len(features)
                    found_tuples+=1

                    #get the preview urls
                    query = "https://api.spotify.com/v1/tracks/?ids=" + ','.join(feature_ids)
                    
                    result_previews  = requests.request('get', query, headers=header).json()['tracks']
                    previews = [r['preview_url'] if (r is not None) else '' for r in result_previews]
                    time.sleep(0.3)
                    
                    #get the covers
                    default_cover = "https://vignette.wikia.nocookie.net/uncyclopedia/images/a/a0/Questionmark.jpg/revision/latest?cb=20090616065802&format=original"
                    covers = []
                    for item in items :
                        if (len(item['album']['images']) != 0) :
                            covers.append(item['album']['images'][0]['url'])
                        else :
                            covers.append(default_cover)    
                    
                    artists = s[1]

                    song_dicts = []
                            
                    for e, f in enumerate(features) :

                        song_dict = {'year' : release,
                                     'artists' : artists,
                                     'name' : track_names[e],
                                     'cover' : covers[e],
                                     'album' : album,
                                     'genre' : genre,
                                    'danceability' : f['danceability'],
                                    'energy' : f['energy'],
                                    'key' : f['key'],
                                    'loudness' : f['loudness'],
                                    'mode' : f['mode'],
                                    'speechiness' : f['speechiness'],
                                    'acousticness' : f['acousticness'],
                                    'instrumentalness' : f['instrumentalness'],
                                    'liveness' : f['liveness'],
                                    'valence' : f['valence'],
                                    'tempo' : f['tempo'],
                                    'duration_ms' : f['duration_ms'],
                                    'preview' : previews[e]}
                        song_dicts.append(song_dict)


                    all_info.extend(song_dicts)
                    
            
                
    #print("found : ", found , ", (" + str(100*found/len(searches)) + "%) for genre " + genre)
    print("found", found, "songs and", found_tuples, "tuples.")
    return all_info

def get_album_features(album_name, artist, album_id) :
    
    #album ids
    album_tracks = requests.request('get', 'https://api.spotify.com/v1/albums/' + album_id + '/tracks', headers=header).json()
    track_ids = [c['id'] for c in album_tracks['items']]
        
    #song features
    features = requests.request('get', 'https://api.spotify.com/v1/audio-features/?ids='+','.join(track_ids), headers=header).json()['audio_features']
    
    aggregated = dict()
    
    for f in all_features :
        aggregated[f] = sum([c[f] for c in features]) / len(features)
    
    aggregated['artists'] = artist
    aggregated['album'] = album_name
    
    return aggregated


# classic albums

In [5]:
json.dump(get_album_features('Control', 'Janet Jackson', '5rgQevCH5hYBGnNkWQwjsI'), open("classic_album_cont.json", 'w'))
json.dump(get_album_features('New Orleans Piano', 'Professor Longhair', '4vQc378UGRZAaJemArhMpv'), open("classic_album_no.json", 'w'))
json.dump(get_album_features('My Blue Heaven', 'Fat Domino', '2P6OFyXNxpIxMZzNlCbrbV'), open("classic_album_early.json", 'w'))
json.dump(get_album_features('Animals', 'The Animals', '3c3VkRXYbjHDeJFiUq8Hse'), open("classic_album_british.json", 'w'))

In [6]:
get_album_features('Control', 'Janet Jackson', '5rgQevCH5hYBGnNkWQwjsI')

{'danceability': 0.7587777777777778,
 'energy': 0.5403333333333333,
 'speechiness': 0.050755555555555554,
 'acousticness': 0.3274444444444444,
 'tempo': 117.24244444444444,
 'instrumentalness': 0.04329497777777777,
 'liveness': 0.1440888888888889,
 'valence': 0.6677777777777778,
 'artists': 'Janet Jackson',
 'album': 'Control'}

# RateYourMusic

### Download Contemporary RnB titles (94 searches, 69 songs found)

From Itunes (69)

In [7]:
# get names from itunes
url = 'https://itunes.apple.com/search?term=song&genreId=1136&limit=200'

response = requests.get(url)
data = response.json()

found = 0

for e in tqdm.tqdm(data['results']):
    result  = requests.request('get', 'https://api.spotify.com/v1/search?q='+ quote(e['trackName']) + "%20artist:" + quote(e['artistName']) + "&type=track", headers=header).json()
    
    items = result['tracks']['items']
    
    
    #We found the song
    if len(items) != 0 and ('error' not in result.keys()):
        
        found += 1
    
        song_id = items[0]['id']    
        features = song_features(song_id)
        
        #get the preview url
        result_preview  = requests.request('get', "https://api.spotify.com/v1/tracks/" + str(song_id), headers=header).json()
        time.sleep(0.3)
        preview = result_preview['preview_url']

        if preview is None :
            preview = ""

        year = e['releaseDate']
        cover = e['artworkUrl100']
        artists = " and ".join([a['name'] for a in result['tracks']['items'][0]['artists']])
        name = e['trackName']
        album = result['tracks']['items'][0]['album']['name']



        song_dict = {'year' : year,
                     'artists' : artists,
                     'name' : name,
                     'cover' : cover,
                     'album' : album,
                     'genre' : "contemporary-rnb",
                    'danceability' : features['danceability'],
                    'energy' : features['energy'],
                    'key' : features['key'],
                    'loudness' : features['loudness'],
                    'mode' : features['mode'],
                    'speechiness' : features['speechiness'],
                    'acousticness' : features['acousticness'],
                    'instrumentalness' : features['instrumentalness'],
                    'liveness' : features['liveness'],
                    'valence' : features['liveness'],
                    'tempo' : features['tempo'],
                    'duration_ms' : features['duration_ms'],
                    'preview' : preview}


        final_json.append(song_dict)
        
intermediary_results = final_json.copy()

100%|██████████| 94/94 [01:07<00:00,  1.18it/s]


### Download british RnB (2*426 searches, 148 tuples found, 1640 songs)

In [13]:
file_names_brit = []
for nb in range(250) :
    file_name = "data/british_rnb/british_rnb_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_brit.append(file_name)

#break the search in two
brit_tuples = get_rateYourMusic_tuples(file_names_brit)    


final_json.extend(search_tuples("british-rnb", brit_tuples[:int(len(brit_tuples)/2)]))
intermediary_results = final_json.copy()

final_json.extend(search_tuples("british-rnb", brit_tuples[int(len(brit_tuples)/2):]))
intermediary_results = final_json.copy()


0it [00:00, ?it/s]
1it [00:00,  1.56it/s]
2it [00:01,  1.57it/s]
3it [00:02,  1.28it/s]
4it [00:03,  1.32it/s]
5it [00:03,  1.37it/s]
6it [00:04,  1.42it/s]
7it [00:05,  1.46it/s]
8it [00:05,  1.47it/s]
9it [00:06,  1.49it/s]
10it [00:07,  1.50it/s]
11it [00:07,  1.51it/s]
12it [00:08,  1.23it/s]
13it [00:09,  1.28it/s]
14it [00:10,  1.34it/s]
15it [00:10,  1.39it/s]
16it [00:11,  1.43it/s]
17it [00:12,  1.45it/s]
18it [00:12,  1.47it/s]
19it [00:13,  1.44it/s]
20it [00:14,  1.45it/s]
21it [00:14,  1.48it/s]
22it [00:15,  1.49it/s]
23it [00:16,  1.50it/s]
24it [00:16,  1.51it/s]
25it [00:17,  1.52it/s]
26it [00:18,  1.51it/s]
27it [00:18,  1.51it/s]
28it [00:19,  1.49it/s]
29it [00:20,  1.19it/s]
30it [00:21,  1.27it/s]
31it [00:22,  1.32it/s]
32it [00:22,  1.38it/s]
33it [00:23,  1.41it/s]
34it [00:24,  1.42it/s]
35it [00:24,  1.45it/s]
36it [00:25,  1.44it/s]
37it [00:26,  1.21it/s]
38it [00:27,  1.09it/s]
39it [00:28,  1.19it/s]
40it [00:29,  1.07it/s]
41it [00:30,  1.17it/s]
42it 

296it [03:45,  1.28it/s]
297it [03:46,  1.12it/s]
298it [03:47,  1.22it/s]
299it [03:48,  1.29it/s]
300it [03:48,  1.34it/s]
301it [03:49,  1.39it/s]
302it [03:50,  1.42it/s]
303it [03:50,  1.46it/s]
304it [03:51,  1.48it/s]
305it [03:52,  1.50it/s]
306it [03:53,  1.14it/s]
307it [03:54,  1.24it/s]
308it [03:54,  1.31it/s]
309it [03:55,  1.37it/s]
310it [03:56,  1.42it/s]
311it [03:56,  1.46it/s]
312it [03:57,  1.48it/s]
313it [03:58,  1.49it/s]
314it [03:58,  1.51it/s]
315it [03:59,  1.52it/s]
316it [04:00,  1.52it/s]
317it [04:00,  1.53it/s]
318it [04:01,  1.51it/s]
319it [04:02,  1.26it/s]
320it [04:03,  1.11it/s]
321it [04:04,  1.03it/s]
322it [04:05,  1.15it/s]
323it [04:06,  1.23it/s]
324it [04:06,  1.31it/s]
325it [04:07,  1.13it/s]
326it [04:08,  1.22it/s]
327it [04:09,  1.31it/s]
328it [04:09,  1.37it/s]
329it [04:11,  1.17it/s]
330it [04:11,  1.26it/s]
331it [04:12,  1.34it/s]
332it [04:13,  1.15it/s]
333it [04:14,  1.25it/s]
334it [04:14,  1.33it/s]
335it [04:15,  1.11it/s]


found 604 songs and 67 tuples.



1it [00:00,  1.57it/s]
2it [00:01,  1.28it/s]
3it [00:02,  1.34it/s]
4it [00:03,  1.39it/s]
5it [00:03,  1.43it/s]
6it [00:04,  1.45it/s]
7it [00:05,  1.20it/s]
8it [00:06,  1.26it/s]
9it [00:06,  1.34it/s]
10it [00:07,  1.40it/s]
11it [00:08,  1.41it/s]
12it [00:08,  1.41it/s]
13it [00:10,  1.16it/s]
14it [00:10,  1.22it/s]
15it [00:12,  1.08it/s]
16it [00:13,  1.02it/s]
17it [00:13,  1.14it/s]
18it [00:14,  1.24it/s]
19it [00:15,  1.09it/s]
20it [00:16,  1.20it/s]
21it [00:16,  1.29it/s]
22it [00:17,  1.32it/s]
23it [00:18,  1.37it/s]
24it [00:19,  1.04it/s]
25it [00:20,  1.08it/s]
26it [00:21,  1.00s/it]
27it [00:22,  1.12it/s]
28it [00:23,  1.22it/s]
29it [00:23,  1.26it/s]
30it [00:24,  1.30it/s]
31it [00:25,  1.36it/s]
32it [00:25,  1.38it/s]
33it [00:26,  1.42it/s]
34it [00:27,  1.20it/s]
35it [00:28,  1.29it/s]
36it [00:28,  1.36it/s]
37it [00:29,  1.39it/s]
38it [00:30,  1.18it/s]
39it [00:31,  1.27it/s]
40it [00:32,  1.35it/s]
41it [00:32,  1.38it/s]
42it [00:33,  1.40it/s]


297it [03:42,  1.38it/s]
298it [03:43,  1.42it/s]
299it [03:44,  1.44it/s]
300it [03:44,  1.46it/s]
301it [03:45,  1.48it/s]
302it [03:46,  1.47it/s]
303it [03:46,  1.46it/s]
304it [03:47,  1.43it/s]
305it [03:48,  1.41it/s]
306it [03:49,  1.07it/s]
307it [03:50,  1.12it/s]
308it [03:51,  1.16it/s]
309it [03:52,  1.21it/s]
310it [03:52,  1.26it/s]
311it [03:53,  1.10it/s]
312it [03:54,  1.17it/s]
313it [03:55,  1.24it/s]
314it [03:56,  1.30it/s]
315it [03:56,  1.35it/s]
316it [03:58,  1.11it/s]
317it [03:58,  1.21it/s]
318it [03:59,  1.29it/s]
319it [04:00,  1.14it/s]
320it [04:01,  1.03it/s]
321it [04:02,  1.14it/s]
322it [04:02,  1.23it/s]
323it [04:03,  1.28it/s]
324it [04:04,  1.11it/s]
325it [04:05,  1.19it/s]
326it [04:06,  1.26it/s]
327it [04:06,  1.33it/s]
328it [04:07,  1.37it/s]
329it [04:08,  1.43it/s]
330it [04:08,  1.47it/s]
331it [04:09,  1.28it/s]
332it [04:11,  1.10it/s]
333it [04:11,  1.21it/s]
334it [04:12,  1.07it/s]
335it [04:14,  1.02s/it]
336it [04:14,  1.10it/s]


found 1036 songs and 81 tuples.


### Download alternative RnB (1217 searches, 640 tuples and 4353 songs)

In [14]:
file_names_alt = []
for nb in range(250) :
    file_name = "data/alternative_rnb/alt_songs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_alt.append(file_name)

alt_tuples = get_rateYourMusic_tuples(file_names_alt)

        
final_json.extend(search_tuples("alternative-rnb", alt_tuples))
intermediary_results = final_json.copy()


0it [00:00, ?it/s]
1it [00:01,  1.07s/it]
2it [00:01,  1.05it/s]
3it [00:02,  1.04s/it]
4it [00:04,  1.11s/it]
5it [00:04,  1.02it/s]
6it [00:06,  1.03s/it]
7it [00:07,  1.07s/it]
8it [00:07,  1.05it/s]
9it [00:08,  1.16it/s]
10it [00:09,  1.25it/s]
11it [00:09,  1.31it/s]
12it [00:10,  1.37it/s]
13it [00:11,  1.40it/s]
14it [00:11,  1.43it/s]
15it [00:12,  1.48it/s]
16it [00:13,  1.21it/s]
17it [00:14,  1.28it/s]
18it [00:15,  1.13it/s]
19it [00:16,  1.24it/s]
20it [00:16,  1.30it/s]
21it [00:17,  1.37it/s]
22it [00:18,  1.16it/s]
23it [00:19,  1.26it/s]
24it [00:19,  1.33it/s]
25it [00:20,  1.39it/s]
26it [00:21,  1.42it/s]
27it [00:21,  1.44it/s]
28it [00:22,  1.47it/s]
29it [00:23,  1.23it/s]
30it [00:24,  1.29it/s]
31it [00:24,  1.37it/s]
32it [00:25,  1.31it/s]
33it [00:26,  1.31it/s]
34it [00:27,  1.37it/s]
35it [00:27,  1.43it/s]
36it [00:28,  1.47it/s]
37it [00:29,  1.50it/s]
38it [00:29,  1.52it/s]
39it [00:30,  1.51it/s]
40it [00:31,  1.53it/s]
41it [00:32,  1.25it/s]
42it 

296it [04:09,  1.14it/s]
297it [04:11,  1.04it/s]
298it [04:11,  1.14it/s]
299it [04:12,  1.03it/s]
300it [04:14,  1.01s/it]
301it [04:14,  1.10it/s]
302it [04:15,  1.01it/s]
303it [04:16,  1.13it/s]
304it [04:17,  1.04it/s]
305it [04:18,  1.02s/it]
306it [04:19,  1.10it/s]
307it [04:20,  1.04s/it]
308it [04:21,  1.08it/s]
309it [04:22,  1.02it/s]
310it [04:23,  1.04s/it]
311it [04:24,  1.08it/s]
312it [04:25,  1.18it/s]
313it [04:25,  1.27it/s]
314it [04:26,  1.12it/s]
315it [04:27,  1.21it/s]
316it [04:28,  1.09it/s]
317it [04:29,  1.00it/s]
318it [04:31,  1.03s/it]
319it [04:31,  1.10it/s]
320it [04:32,  1.02it/s]
321it [04:33,  1.03s/it]
322it [04:35,  1.09s/it]
323it [04:35,  1.05it/s]
324it [04:36,  1.15it/s]
325it [04:37,  1.06it/s]
326it [04:38,  1.16it/s]
327it [04:38,  1.25it/s]
328it [04:40,  1.11it/s]
329it [04:41,  1.03it/s]
330it [04:42,  1.02s/it]
331it [04:43,  1.04s/it]
332it [04:44,  1.08it/s]
333it [04:44,  1.20it/s]
334it [04:45,  1.29it/s]
335it [04:45,  1.36it/s]


588it [08:37,  1.12it/s]
589it [08:38,  1.03it/s]
590it [08:39,  1.02s/it]
591it [08:40,  1.06s/it]
592it [08:41,  1.09s/it]
593it [08:43,  1.10s/it]
594it [08:44,  1.11s/it]
595it [08:44,  1.03it/s]
596it [08:45,  1.02s/it]
597it [08:47,  1.06s/it]
598it [08:48,  1.08s/it]
599it [08:49,  1.14s/it]
600it [08:50,  1.00it/s]
601it [08:51,  1.03s/it]
602it [08:51,  1.09it/s]
603it [08:53,  1.02it/s]
604it [08:53,  1.12it/s]
605it [08:54,  1.02it/s]
606it [08:55,  1.13it/s]
607it [08:56,  1.05it/s]
608it [08:57,  1.15it/s]
609it [08:58,  1.05it/s]
610it [08:59,  1.16it/s]
611it [09:00,  1.06it/s]
612it [09:00,  1.17it/s]
613it [09:01,  1.27it/s]
614it [09:02,  1.11it/s]
615it [09:03,  1.18it/s]
616it [09:04,  1.28it/s]
617it [09:05,  1.12it/s]
618it [09:05,  1.22it/s]
619it [09:07,  1.09it/s]
620it [09:07,  1.20it/s]
621it [09:08,  1.06it/s]
622it [09:09,  1.18it/s]
623it [09:10,  1.08it/s]
624it [09:11,  1.18it/s]
625it [09:12,  1.07it/s]
626it [09:13,  1.00s/it]
627it [09:14,  1.04s/it]


880it [13:13,  1.12it/s]
881it [13:14,  1.03it/s]
882it [13:15,  1.12it/s]
883it [13:15,  1.22it/s]
884it [13:16,  1.29it/s]
885it [13:17,  1.13it/s]
886it [13:18,  1.22it/s]
887it [13:19,  1.11it/s]
888it [13:20,  1.20it/s]
889it [13:20,  1.28it/s]
890it [13:21,  1.12it/s]
891it [13:23,  1.02it/s]
892it [13:24,  1.03s/it]
893it [13:24,  1.09it/s]
894it [13:25,  1.19it/s]
895it [13:26,  1.28it/s]
896it [13:27,  1.13it/s]
897it [13:28,  1.03it/s]
898it [13:29,  1.02s/it]
899it [13:30,  1.10it/s]
900it [13:31,  1.20it/s]
901it [13:31,  1.29it/s]
902it [13:32,  1.34it/s]
903it [13:33,  1.15it/s]
904it [13:34,  1.06it/s]
905it [13:35,  1.18it/s]
906it [13:35,  1.27it/s]
907it [13:37,  1.12it/s]
908it [13:38,  1.03it/s]
909it [13:38,  1.14it/s]
910it [13:39,  1.05it/s]
911it [13:41,  1.01s/it]
912it [13:41,  1.10it/s]
913it [13:42,  1.03it/s]
914it [13:44,  1.03s/it]
915it [13:45,  1.06s/it]
916it [13:46,  1.07s/it]
917it [13:47,  1.10s/it]
918it [13:48,  1.13s/it]
919it [13:49,  1.14s/it]


1166it [17:52,  1.03it/s]
1167it [17:53,  1.00s/it]
1168it [17:54,  1.04s/it]
1169it [17:55,  1.09it/s]
1170it [17:56,  1.02it/s]
1171it [17:57,  1.03s/it]
1172it [17:59,  1.09s/it]
1173it [18:00,  1.11s/it]
1174it [18:00,  1.02it/s]
1175it [18:01,  1.14it/s]
1176it [18:02,  1.05it/s]
1177it [18:03,  1.16it/s]
1178it [18:04,  1.07it/s]
1179it [18:05,  1.01it/s]
1180it [18:06,  1.03s/it]
1181it [18:07,  1.06s/it]
1182it [18:08,  1.08s/it]
1183it [18:09,  1.09s/it]
1184it [18:10,  1.04it/s]
1185it [18:11,  1.15it/s]
1186it [18:12,  1.06it/s]
1187it [18:13,  1.18it/s]
1188it [18:13,  1.27it/s]
1189it [18:14,  1.13it/s]
1190it [18:15,  1.05it/s]
1191it [18:16,  1.15it/s]
1192it [18:17,  1.06it/s]
1193it [18:18,  1.01s/it]
1194it [18:19,  1.05s/it]
1195it [18:21,  1.08s/it]
1196it [18:22,  1.08s/it]
1197it [18:22,  1.04it/s]
1198it [18:24,  1.01s/it]
1199it [18:25,  1.05s/it]
1200it [18:26,  1.07s/it]
1201it [18:27,  1.03it/s]
1202it [18:28,  1.02s/it]
1203it [18:28,  1.09it/s]
1204it [18:2

found 4353 songs and 640 tuples.


### Download new jack swing RnB (560 searches, 146 tuples and 1764 songs)

In [15]:
file_names_nj = []
for nb in range(250) :
    file_name = "data/new_jack_swing_rnb/njs_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_nj.append(file_name)

njs_tuples = get_rateYourMusic_tuples(file_names_nj)
        
final_json.extend(search_tuples("new-jack-swing-rnb", njs_tuples))
intermediary_results = final_json.copy()


0it [00:00, ?it/s]
1it [00:00,  1.56it/s]
2it [00:01,  1.55it/s]
3it [00:01,  1.55it/s]
4it [00:03,  1.24it/s]
5it [00:03,  1.31it/s]
6it [00:04,  1.38it/s]
7it [00:05,  1.18it/s]
8it [00:06,  1.25it/s]
9it [00:06,  1.32it/s]
10it [00:07,  1.38it/s]
11it [00:08,  1.43it/s]
12it [00:08,  1.46it/s]
13it [00:10,  1.19it/s]
14it [00:10,  1.26it/s]
15it [00:11,  1.10it/s]
16it [00:12,  1.20it/s]
17it [00:13,  1.05it/s]
18it [00:14,  1.15it/s]
19it [00:15,  1.05it/s]
20it [00:16,  1.15it/s]
21it [00:16,  1.24it/s]
22it [00:17,  1.32it/s]
23it [00:18,  1.36it/s]
24it [00:19,  1.15it/s]
25it [00:20,  1.25it/s]
26it [00:20,  1.31it/s]
27it [00:21,  1.35it/s]
28it [00:22,  1.40it/s]
29it [00:22,  1.43it/s]
30it [00:23,  1.44it/s]
31it [00:24,  1.44it/s]
32it [00:24,  1.47it/s]
33it [00:25,  1.48it/s]
34it [00:26,  1.49it/s]
35it [00:27,  1.22it/s]
36it [00:27,  1.29it/s]
37it [00:29,  1.14it/s]
38it [00:29,  1.22it/s]
39it [00:30,  1.28it/s]
40it [00:31,  1.12it/s]
41it [00:32,  1.19it/s]
42it 

296it [04:09,  1.03s/it]
297it [04:10,  1.10s/it]
298it [04:11,  1.02it/s]
299it [04:12,  1.11it/s]
300it [04:13,  1.00it/s]
301it [04:14,  1.09it/s]
302it [04:14,  1.17it/s]
303it [04:15,  1.23it/s]
304it [04:16,  1.27it/s]
305it [04:16,  1.31it/s]
306it [04:17,  1.33it/s]
307it [04:18,  1.35it/s]
308it [04:19,  1.36it/s]
309it [04:19,  1.37it/s]
310it [04:21,  1.15it/s]
311it [04:22,  1.02it/s]
312it [04:23,  1.06s/it]
313it [04:24,  1.12s/it]
314it [04:25,  1.01it/s]
315it [04:26,  1.05s/it]
316it [04:27,  1.04it/s]
317it [04:28,  1.13it/s]
318it [04:28,  1.19it/s]
319it [04:29,  1.25it/s]
320it [04:30,  1.29it/s]
321it [04:31,  1.10it/s]
322it [04:32,  1.17it/s]
323it [04:33,  1.04it/s]
324it [04:34,  1.03s/it]
325it [04:35,  1.08it/s]
326it [04:36,  1.01s/it]
327it [04:37,  1.08it/s]
328it [04:37,  1.16it/s]
329it [04:38,  1.18it/s]
330it [04:39,  1.23it/s]
331it [04:40,  1.28it/s]
332it [04:40,  1.31it/s]
333it [04:41,  1.33it/s]
334it [04:42,  1.35it/s]
335it [04:43,  1.36it/s]


found 1764 songs and 146 tuples.


### Download new Orleans RnB  (1087 searches, 245 tuples and 3014 songs)

In [18]:
file_names_no = []
for nb in range(250) :
    file_name = "data/new_orleans_rnb/no_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_no.append(file_name)
        
no_tuples = get_rateYourMusic_tuples(file_names_no)

final_json.extend(search_tuples("new-orleans-rnb", no_tuples))
intermediary_results = final_json.copy()



0it [00:00, ?it/s]

1it [00:00,  1.46it/s]

2it [00:01,  1.44it/s]

3it [00:02,  1.46it/s]

4it [00:02,  1.41it/s]

5it [00:04,  1.17it/s]

6it [00:04,  1.21it/s]

7it [00:05,  1.26it/s]

8it [00:06,  1.30it/s]

9it [00:07,  1.12it/s]

10it [00:08,  1.17it/s]

11it [00:08,  1.23it/s]

12it [00:09,  1.27it/s]

13it [00:10,  1.31it/s]

14it [00:11,  1.12it/s]

15it [00:12,  1.02it/s]

16it [00:13,  1.04s/it]

17it [00:14,  1.05it/s]

18it [00:15,  1.14it/s]

19it [00:16,  1.20it/s]

20it [00:16,  1.25it/s]

21it [00:17,  1.30it/s]

22it [00:18,  1.32it/s]

23it [00:18,  1.34it/s]

24it [00:19,  1.36it/s]

25it [00:20,  1.15it/s]

26it [00:21,  1.19it/s]

27it [00:22,  1.25it/s]

28it [00:23,  1.29it/s]

29it [00:24,  1.11it/s]

30it [00:24,  1.17it/s]

31it [00:26,  1.12s/it]

32it [00:27,  1.00s/it]

33it [00:28,  1.09it/s]

34it [00:28,  1.17it/s]

35it [00:29,  1.23it/s]

36it [00:30,  1.27it/s]

37it [00:30,  1.31it/s]

38it [00:31,  1.33it/s]

39it [00:32,  1.35it/s]

40it [00:33,

259it [03:28,  1.33it/s]

260it [03:29,  1.32it/s]

261it [03:30,  1.08it/s]

262it [03:31,  1.15it/s]

263it [03:32,  1.21it/s]

264it [03:32,  1.26it/s]

265it [03:33,  1.30it/s]

266it [03:34,  1.33it/s]

267it [03:34,  1.35it/s]

268it [03:36,  1.13it/s]

269it [03:37,  1.03it/s]

270it [03:38,  1.07s/it]

271it [03:39,  1.11s/it]

272it [03:40,  1.03it/s]

273it [03:41,  1.04s/it]

274it [03:42,  1.06it/s]

275it [03:43,  1.14it/s]

276it [03:43,  1.20it/s]

277it [03:44,  1.26it/s]

278it [03:45,  1.29it/s]

279it [03:45,  1.32it/s]

280it [03:46,  1.34it/s]

281it [03:47,  1.38it/s]

282it [03:48,  1.42it/s]

283it [03:49,  1.20it/s]

284it [03:49,  1.26it/s]

285it [03:50,  1.30it/s]

286it [03:51,  1.32it/s]

287it [03:52,  1.34it/s]

288it [03:52,  1.31it/s]

289it [03:53,  1.33it/s]

290it [03:54,  1.35it/s]

291it [03:54,  1.36it/s]

292it [03:56,  1.15it/s]

293it [03:56,  1.20it/s]

294it [03:57,  1.20it/s]

295it [03:58,  1.17it/s]

296it [03:59,  1.23it/s]

297it [04:00

515it [06:59,  1.35it/s]

516it [06:59,  1.36it/s]

517it [07:00,  1.37it/s]

518it [07:01,  1.37it/s]

519it [07:01,  1.38it/s]

520it [07:02,  1.39it/s]

521it [07:03,  1.39it/s]

522it [07:04,  1.16it/s]

523it [07:05,  1.25it/s]

524it [07:05,  1.30it/s]

525it [07:06,  1.32it/s]

526it [07:07,  1.38it/s]

527it [07:08,  1.41it/s]

528it [07:08,  1.45it/s]

529it [07:09,  1.45it/s]

530it [07:10,  1.43it/s]

531it [07:10,  1.42it/s]

532it [07:11,  1.41it/s]

533it [07:12,  1.17it/s]

534it [07:13,  1.26it/s]

535it [07:14,  1.30it/s]

536it [07:14,  1.33it/s]

537it [07:15,  1.38it/s]

538it [07:16,  1.13it/s]

539it [07:17,  1.19it/s]

540it [07:18,  1.24it/s]

541it [07:18,  1.24it/s]

542it [07:19,  1.28it/s]

543it [07:20,  1.31it/s]

544it [07:21,  1.34it/s]

545it [07:22,  1.13it/s]

546it [07:22,  1.23it/s]

547it [07:23,  1.32it/s]

548it [07:24,  1.39it/s]

549it [07:24,  1.44it/s]

550it [07:25,  1.40it/s]

551it [07:26,  1.40it/s]

552it [07:27,  1.40it/s]

553it [07:27

771it [10:41,  1.23it/s]

772it [10:42,  1.31it/s]

773it [10:43,  1.09it/s]

774it [10:44,  1.16it/s]

775it [10:45,  1.22it/s]

776it [10:46,  1.27it/s]

777it [10:46,  1.31it/s]

778it [10:47,  1.33it/s]

779it [10:48,  1.11it/s]

780it [10:49,  1.19it/s]

781it [10:50,  1.25it/s]

782it [10:51,  1.06it/s]

783it [10:52,  1.10s/it]

784it [10:54,  1.14s/it]

785it [10:55,  1.17s/it]

786it [10:56,  1.04s/it]

787it [10:56,  1.06it/s]

788it [10:57,  1.14it/s]

789it [10:58,  1.02it/s]

790it [10:59,  1.05s/it]

791it [11:01,  1.14s/it]

792it [11:01,  1.01s/it]

793it [11:03,  1.09s/it]

794it [11:03,  1.04it/s]

795it [11:04,  1.12it/s]

796it [11:05,  1.19it/s]

797it [11:06,  1.25it/s]

798it [11:06,  1.29it/s]

799it [11:07,  1.32it/s]

800it [11:08,  1.11it/s]

801it [11:09,  1.18it/s]

802it [11:10,  1.24it/s]

803it [11:10,  1.24it/s]

804it [11:12,  1.08it/s]

805it [11:12,  1.14it/s]

806it [11:13,  1.21it/s]

807it [11:14,  1.26it/s]

808it [11:15,  1.30it/s]

809it [11:16

1026it [14:28,  1.05it/s]

1027it [14:29,  1.12it/s]

1028it [14:30,  1.00it/s]

1029it [14:31,  1.07s/it]

1030it [14:32,  1.05it/s]

1031it [14:33,  1.14it/s]

1032it [14:34,  1.02it/s]

1033it [14:35,  1.04s/it]

1034it [14:36,  1.07it/s]

1035it [14:37,  1.15it/s]

1036it [14:37,  1.21it/s]

1037it [14:39,  1.06it/s]

1038it [14:40,  1.03s/it]

1039it [14:41,  1.07it/s]

1040it [14:41,  1.15it/s]

1041it [14:42,  1.21it/s]

1042it [14:43,  1.06it/s]

1043it [14:44,  1.14it/s]

1044it [14:45,  1.20it/s]

1045it [14:45,  1.26it/s]

1046it [14:46,  1.30it/s]

1047it [14:47,  1.32it/s]

1048it [14:48,  1.11it/s]

1049it [14:49,  1.00s/it]

1050it [14:50,  1.08s/it]

1051it [14:51,  1.01it/s]

1052it [14:52,  1.10it/s]

1053it [14:53,  1.18it/s]

1054it [14:54,  1.05it/s]

1055it [14:55,  1.12it/s]

1056it [14:55,  1.19it/s]

1057it [14:56,  1.20it/s]

1058it [14:57,  1.25it/s]

1059it [14:58,  1.29it/s]

1060it [14:58,  1.32it/s]

1061it [14:59,  1.34it/s]

1062it [15:00,  1.36it/s]

1

found 3014 songs and 245 tuples.


### Download Jump Blues (718 searches, 161 tuples and 2094 songs)

In [19]:
file_names_jb = []
for nb in range(250) :
    file_name = "data/jump_blues_rnb/jp_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_jb.append(file_name)

jb_tuples = get_rateYourMusic_tuples(file_names_jb)
        
final_json.extend(search_tuples("jump-blues", jb_tuples))
intermediary_results = final_json.copy()



0it [00:00, ?it/s]

1it [00:01,  1.30s/it]

2it [00:02,  1.26s/it]

3it [00:03,  1.11s/it]

4it [00:03,  1.01it/s]

5it [00:04,  1.10it/s]

6it [00:05,  1.18it/s]

7it [00:06,  1.24it/s]

8it [00:06,  1.28it/s]

9it [00:07,  1.33it/s]

10it [00:08,  1.33it/s]

11it [00:08,  1.35it/s]

12it [00:09,  1.31it/s]

13it [00:10,  1.35it/s]

14it [00:11,  1.35it/s]

15it [00:11,  1.40it/s]

16it [00:12,  1.36it/s]

17it [00:13,  1.37it/s]

18it [00:14,  1.38it/s]

19it [00:14,  1.38it/s]

20it [00:15,  1.39it/s]

21it [00:16,  1.39it/s]

22it [00:16,  1.39it/s]

23it [00:17,  1.39it/s]

24it [00:18,  1.39it/s]

25it [00:19,  1.34it/s]

26it [00:20,  1.11it/s]

27it [00:21,  1.04s/it]

28it [00:22,  1.05it/s]

29it [00:23,  1.13it/s]

30it [00:23,  1.20it/s]

31it [00:24,  1.25it/s]

32it [00:25,  1.29it/s]

33it [00:26,  1.32it/s]

34it [00:26,  1.34it/s]

35it [00:27,  1.36it/s]

36it [00:28,  1.37it/s]

37it [00:29,  1.38it/s]

38it [00:29,  1.38it/s]

39it [00:30,  1.39it/s]

40it [00:31,

259it [03:22,  1.26it/s]

260it [03:24,  1.08it/s]

261it [03:24,  1.18it/s]

262it [03:25,  1.24it/s]

263it [03:26,  1.30it/s]

264it [03:26,  1.35it/s]

265it [03:28,  1.17it/s]

266it [03:28,  1.24it/s]

267it [03:29,  1.31it/s]

268it [03:30,  1.35it/s]

269it [03:30,  1.33it/s]

270it [03:31,  1.39it/s]

271it [03:32,  1.42it/s]

272it [03:32,  1.44it/s]

273it [03:33,  1.45it/s]

274it [03:34,  1.45it/s]

275it [03:34,  1.47it/s]

276it [03:35,  1.48it/s]

277it [03:36,  1.47it/s]

278it [03:36,  1.47it/s]

279it [03:37,  1.47it/s]

280it [03:38,  1.49it/s]

281it [03:38,  1.48it/s]

282it [03:39,  1.49it/s]

283it [03:40,  1.18it/s]

284it [03:41,  1.27it/s]

285it [03:43,  1.19s/it]

286it [03:44,  1.03s/it]

287it [03:45,  1.10s/it]

288it [03:46,  1.03it/s]

289it [03:47,  1.04s/it]

290it [03:48,  1.08it/s]

291it [03:49,  1.02it/s]

292it [03:50,  1.03s/it]

293it [03:51,  1.07it/s]

294it [03:51,  1.17it/s]

295it [03:52,  1.26it/s]

296it [03:53,  1.11it/s]

297it [03:54

515it [06:42,  1.28it/s]

516it [06:43,  1.07it/s]

517it [06:44,  1.01s/it]

518it [06:45,  1.11it/s]

519it [06:46,  1.21it/s]

520it [06:46,  1.28it/s]

521it [06:48,  1.13it/s]

522it [06:48,  1.23it/s]

523it [06:49,  1.31it/s]

524it [06:50,  1.37it/s]

525it [06:51,  1.18it/s]

526it [06:51,  1.26it/s]

527it [06:52,  1.33it/s]

528it [06:53,  1.39it/s]

529it [06:53,  1.43it/s]

530it [06:54,  1.18it/s]

531it [06:56,  1.06it/s]

532it [06:57,  1.01s/it]

533it [06:58,  1.07s/it]

534it [06:59,  1.10s/it]

535it [07:00,  1.14s/it]

536it [07:01,  1.01it/s]

537it [07:02,  1.12it/s]

538it [07:02,  1.21it/s]

539it [07:03,  1.29it/s]

540it [07:04,  1.35it/s]

541it [07:04,  1.40it/s]

542it [07:05,  1.43it/s]

543it [07:06,  1.45it/s]

544it [07:07,  1.19it/s]

545it [07:08,  1.25it/s]

546it [07:08,  1.32it/s]

547it [07:09,  1.38it/s]

548it [07:10,  1.42it/s]

549it [07:10,  1.44it/s]

550it [07:11,  1.19it/s]

551it [07:13,  1.07it/s]

552it [07:14,  1.00s/it]

553it [07:15

found 2094 songs and 161 tuples.


In [21]:
json.dump(intermediary_results, open("start_at_blue_eyed.json", 'w'))

12934

### Download blue-eyed soul (622 songs)

In [20]:
file_names_bes = []
for nb in range(250) :
    file_name = "data/blue_eyed_soul/blue_eyed_soul_" + str(nb) + ".html"
    if os.path.isfile(file_name) :
        file_names_bes.append(file_name)

bes_tuples = get_rateYourMusic_tuples(file_names_bes)
        
final_json.extend(search_tuples("blue-eyed-soul", bes_tuples))



0it [00:00, ?it/s]

1it [00:01,  1.14s/it]

2it [00:01,  1.01it/s]

3it [00:02,  1.13it/s]

4it [00:03,  1.23it/s]

5it [00:04,  1.06it/s]

6it [00:04,  1.16it/s]

7it [00:05,  1.23it/s]

8it [00:06,  1.31it/s]

9it [00:07,  1.32it/s]

10it [00:08,  1.10it/s]

11it [00:09,  1.19it/s]

12it [00:10,  1.03it/s]

13it [00:11,  1.02s/it]

14it [00:12,  1.09it/s]

15it [00:12,  1.19it/s]

16it [00:13,  1.21it/s]

17it [00:14,  1.09it/s]

18it [00:15,  1.02s/it]

19it [00:16,  1.10it/s]

20it [00:17,  1.21it/s]

21it [00:17,  1.30it/s]

22it [00:19,  1.11it/s]

23it [00:20,  1.03it/s]

24it [00:20,  1.13it/s]

25it [00:21,  1.22it/s]

26it [00:22,  1.08it/s]

27it [00:23,  1.18it/s]

28it [00:24,  1.25it/s]

29it [00:24,  1.33it/s]

30it [00:25,  1.39it/s]

31it [00:26,  1.44it/s]

32it [00:26,  1.45it/s]

33it [00:27,  1.47it/s]

34it [00:28,  1.20it/s]

35it [00:29,  1.08it/s]

36it [00:30,  1.18it/s]

KeyboardInterrupt: 

In [ ]:
#Dump everything into the data
json.dump(final_json, open("rnb_data.json", 'w'))

# AllMusic

### Contemporary rnb (720)

In [ ]:
cont_tuples = json.load(open("data/All_Music/parsed/am_contemporary_rnb.json"))
final_json.extend(search_tuples("contemporary-rnb", cont_tuples))
intermediary_results = final_json.copy()

### early rnb (198)

In [ ]:
early_tuples = json.load(open("data/All_Music/parsed/am_early_rnb.json"))
final_json.extend(search_tuples("early-rnb", early_tuples))
intermediary_results = final_json.copy()

### motown (86)

In [ ]:
mo_tuples = json.load(open("data/All_Music/parsed/am_motown.json"))
final_json.extend(search_tuples("motown-rnb", mo_tuples))
intermediary_results = final_json.copy()

In [ ]:
json.dump(final_json, open("rnb_data.json", 'w'))

# visualize number of songs per year

In [ ]:
import collections

#all possible genres
genres = ['contemporary-rnb', 'british-rnb', 'new-orleans-rnb', 
          'alternative-rnb', 'new-jack-swing-rnb', 'blue-eyed-soul', 'jump-blues']

#compute earliest and latest year globally
all_years = [d['year'] for d in final_json if (d['year'] != '')]
min_all_years, max_all_years = int(min(all_years)), int(max(all_years)) + 1

#plot settings
width = 1
figure(figsize=(15,10))
plots = []

#initialize all years to zero
seen_years = collections.Counter()
for y in range(min_all_years, max_all_years) :
        seen_years[str(y)] = 0



for g in genres :

    #years of genre songs
    genre_years = [d['year'] for d in final_json if ((d["genre"] == g) and (d['year'] != ''))]
    
    #create counter
    years_per_genre = collections.Counter()
    
    
    #put 0 as default value for all years
    for y in range(min_all_years, max_all_years) :
        years_per_genre[str(y)] = 0
        
    #count values
    years_per_genre.update(genre_years)
    
    #create plot variables
    labels, values = zip(*years_per_genre.items())
    labels_all, values_all = zip(*seen_years.items())
    indexes = np.arange(len(labels))    
    
    #plot with bottom
    p = plt.bar(indexes, values, width, bottom=values_all)    
    plots.append(p)
    
    #update seen years counter
    seen_years.update(genre_years)

#settings for axes
plt.xticks(indexes + width * 0.5, labels, rotation=90)
plt.xlabel("release year")
plt.ylabel("number of songs")
plt.ylim(ymax = 160, ymin = 0)

#general plot settings
plt.title("number of RnB songs per genre and per year")
plt.legend(plots, genres)

# plot
plt.savefig("nb_of_songs.png")
plt.show()


# compute discriminative coefficients for each feature and each year 

In [ ]:
[f for f in final_json_bis if f['genre'] == 'contemporary-rnb']

In [ ]:
years = range(1900, 2050)
best_features = dict()

for y in years :
    
    # load data
    names = ['energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    targets = np.array([c['genre'] for c in final_json if c['year'] == str(y)])
    
    if len(set(targets)) > 1:

        features = np.array([c['danceability'] for c in final_json if c['year'] == str(y)])
        
        
        for n in names :
            features = np.vstack((features, np.array([c[n] for c in final_json if c['year'] == str(y)])))


        model = LogisticRegression()
        rfe = RFE(model, 2)
        fit = rfe.fit(features.T, targets)

        best_for_year = sorted([c[0] for c in zip(['danceability'] + names, fit.support_) if c[1]])
        best_features[y] = best_for_year
        
best_features